Parcours en largeur
===================

Dans ce notebook, on va s'appuyer sur le type Pile pour construire un type File, puis se servir de ce type File pour construire une méthode BFS de la classe ABR implémentant un parcours en largeur.

Deux pages intéressantes illustrant les notions de <a href="https://alainbusser.frama.io/NSI-IREMI-974/stacksortable.html">piles</a> et de <a href="https://alainbusser.frama.io/NSI-IREMI-974/queuesortable.html
">files</a> sur la thématique du tri.

Une pile est une structure **LIFO** last-in first-out tandis qu'une file est une structure **FIFO** first-in first-out. 

In [1]:
class Pile:
    def __init__(self):
        self.p=[]
    def vide(self):
        return self.p==[]
    def empile(self,e):
        self.p.append(e)
    def depile(self):
        return self.p.pop()

Les primitives associées au type File sont :
* la création d'une file vide
* le test d'une file pour savoir si elle est vide
* enfiler un élément (au bout de la file)
* défiler un élément (au début de la file)

Une des implémentations possibles pour une File est d'utiliser deux piles, une pile d'entrée `E` et une pile de sortie `S`. Les détails de cette implémentation sont explicités en commentaires.

In [2]:
class File:
    def __init__(self):
        # on crée deux piles E et S
        self.E=Pile()
        self.S=Pile()
    def vide(self):
        # la file est vide si les deux piles E et S sont vides
        return self.E.vide() and self.S.vide()
    def enfile(self,e):
        # on empile l'élément sur la pile d'entrée
        self.E.empile(e)
    def defile(self):
        # si la pile de sortie est vide, 
        # on dépile tous les éléments de la pile d'entrée et on les empile sur la pile de sortie
        if self.S.vide():
            while not self.E.vide():
                self.S.empile(self.E.depile())
        # ensuite on renvoie le TOS de la pile de sortie
        return self.S.depile()

**q1** compléter la classe File en suivant les indications données en commentaires

In [3]:
# testons cette classe
f=File()
f.enfile(1)
f.enfile(2)
print(f.vide())
print(f.defile())
f.enfile(3)
f.enfile(4)
while not f.vide():
    print(f.defile())

False
1
2
3
4


**q2** tester la classe en exécutant la cellule ci-dessus ; on commencera par déterminer les sorties attendues, 

afin de pouvoir comparer avec les sorties effectivement affichées

On va utiliser une structure de file dans une méthode d'une classe arbre (binaire de recherche),
afin de parcourir cet arbre **en largeur**. L'algorithme de ce parcours en largeur est en général nommé **BFS** pour Breadth First Search.



In [4]:
class Abr:
    def __init__(self,racine):
        self.r=racine
        self.g=None
        self.d=None
#---------------------------------------------------------------------------
    def racine(self):
        return self.r
#---------------------------------------------------------------------------
    def feuille(self):
        return self.g==None and self.d==None
#---------------------------------------------------------------------------
    def sabg(self):
        return self.g
#---------------------------------------------------------------------------
    def sabd(self):
        return self.d
#---------------------------------------------------------------------------
    def hauteur(self):
        if self.g:
            g=self.g.hauteur()
        else:
            g=-1
        if self.d:
            d=self.d.hauteur()
        else:
            d=-1
        return 1+max(g,d)
#---------------------------------------------------------------------------
    def taille(self):
        t=1
        if self.g:
            t=t+self.g.taille()
        if self.d:
            t=t+self.d.taille()
        return t 
#---------------------------------------------------------------------------
    def insere(self,e):
        if e<self.r:
            if self.g==None:
                self.g=Abr(e)
            else:
                self.g.insere(e)
        elif e>self.r:
            if self.d==None:
                self.d=Abr(e)
            else:
                self.d.insere(e)
#---------------------------------------------------------------------------                
    def recherche(self,e):
        if e == self.r: # si e est égal a la racine
            return True
        elif e < self.r and self.g != None: # si e est inférieur a la racine alors on cherche dans le sous arbre gauche
            return self.g.recherche(e)
        elif e > self.r and self.d != None: # si e est supérieur a la racine alors on cherche dans le sous arbre droit
            return self.d.recherche(e)
        else: # sinon e n'est pas dans l'abre
            return False
        
#    def recherche(self,e):
#        if e==self.r:
#            return True
#        if e<self.r:
#            if self.g is None:
#                return False
#            else:
#                return self.g.recherche(e)
#        if self.d is None:
#            return False
#        else:
#            return self.d.recherche(e)
#---------------------------------------------------------------------------        
    def infixe(self):
        if self.g is None:
            gauche=[]
        else:
            gauche=self.g.infixe()
        if self.d is None:
            droite=[]
        else:
            droite=self.d.infixe()
        return gauche+[self.r]+droite
#---------------------------------------------------------------------------
    def suffixe(self):
        if self.g is None:
            gauche=[]
        else:
            gauche=self.g.suffixe()
        if self.d is None:
            droite=[]
        else:
            droite=self.d.suffixe()
        return gauche+droite+[self.r]
#---------------------------------------------------------------------------   
    def prefixe(self):
        if self.g is None:
            gauche=[]
        else:
            gauche=self.g.prefixe()
        if self.d is None:
            droite=[]
        else:
            droite=self.d.prefixe()
        return [self.r]+gauche+droite
#---------------------------------------------------------------------------    
    def bfs(self):
        f=File()
        # y enfiler l'abr
        f.enfile(self) # self est un abr
        # créer une liste vide 'déjàvu'
        dejavu=[]
        # tant que la file n'est pas vide
        while not f.vide():
            # défiler un sabr
            s=f.defile() # s un (sous)abr
            # ajouter sous étiquette en fin de la liste déjàvu
            dejavu.append(s.racine())
            # enfiler les éventuels fils de ce sommet
            if s.sabg() is not None:
                f.enfile(s.sabg())
            if s.sabd() is not None:
                f.enfile(s.sabd())
        # renvoyer déjàvu
        return dejavu
        

In [5]:
a=Abr(10)
for k in [6,3,1,4,7,14,11,17]:
    a.insere(k)

In [6]:
a.infixe()

[1, 3, 4, 6, 7, 10, 11, 14, 17]

In [7]:
a.suffixe()

[1, 4, 3, 7, 6, 11, 17, 14, 10]

In [8]:
a.prefixe()

[10, 6, 3, 1, 4, 7, 14, 11, 17]

In [9]:
a.bfs()

[10, 6, 14, 3, 7, 11, 17, 1, 4]

In [10]:
a.recherche(17)

True

In [11]:
a.recherche(19)

False

In [12]:
a.hauteur()

3

In [13]:
a.taille()

9

- [x] **q3** dessiner sur papier l'ABR obtenu avec les instructions de la cellule ci-dessus.

- [x] **q4** donner sa hauteur, sa taille, et le résultat que devra renvoyer bfs

- [x] **q5** compléter les méthodes `hauteur`, `taille` et `insere`, et les tester

- [x] **q6** compléter la méthode `bfs`, et la tester

- [x] **q7** compléter la méthode `recherche`, et la tester 

(on prendra soin de chercher un élément dont on sait qu'il est présent dans l'arbre, puis un dont on sait qu'il est absent de l'arbre)

- [x] **q8** écrire trois méthodes `dfs_prefixe`, `dfs_infixe`, et `dfs_postfixe` qui réalisent des parcours de l'arbre en profondeur (Depth First Search), respectivement préfixe, infixe et postfixe.

**q9** quelles seraient les modifications à apporter aux différentes méthodes pour qu'elles puissent s'appliquer à n'importe quel arbre, pas nécessairement de recherche, ni même binaire ?

**q10** refaire tout le TP sans POO